In [1]:
from elasticsearch import Elasticsearch
import os
import py_vncorenlp
import json
import re

In [2]:
vncorenlp_segmentor = py_vncorenlp.VnCoreNLP(max_heap_size='-Xmx8g', annotators=['wseg'], save_dir='/workspace/nlplab/kienvt/scada-tokenize-server/vncorenlp')

2024-03-21 16:56:21 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [3]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import torch

# tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base-v2', cache_dir ='/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/hf_cache')
# model = AutoModel.from_pretrained('vinai/phobert-base-v2', cache_dir ='/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/hf_cache').to('cuda:0')

model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder', 
                            cache_folder='/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/hf_cache').to('cuda:0')

/workspace/nlplab/kienvt/KLTN/kenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from sentence_transformers import SentenceTransformer, models, evaluation
word_embedding_model = models.Transformer('bigscience-catalogue-lm-data/sgpt-nli-bloom-1b3', max_seq_length=256, cache_dir='./hf_cache')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model2 = SentenceTransformer(modules=[word_embedding_model, pooling_model]).to('cuda:0')

In [7]:
model2.encode('Tôi là học sinh trường Đại học Công Nghệ').shape

(2048,)

In [4]:
ELASTIC_API_KEY = os.getenv('ELASTIC_API_KEY', 'WnpxRjc0MEJ0OUFnM2g4V216amE6STh3aDNnd1lTd1dGdS1FMER5enVOUQ==')

In [5]:
client = Elasticsearch('http://localhost:9200', api_key=ELASTIC_API_KEY)

In [14]:
settings = {
    "number_of_shards": 1,
    "index" : {
        "similarity" : {
            "default" : {
                "type" : "BM25",
                "b": 0.75,
                "k1": 1.2,
            }
        }
    }
}

mappings = {
    "properties": {
        "content_vector": {
            "type": "dense_vector",
            "dims": 768,
            "index": "true",
            "similarity": "cosine",
        }
    }
}


# client.indices.create(index='vn-register-webportal-v2', settings=settings)
# client.indices.create(index='vn-register-webportal-embedding-v2', settings=settings, mappings=mappings)
# client.indices.create(index='vn-register-webportal-segment', settings=settings)
# client.indices.create(index='vn-register-webportal-segment-embedding', settings=settings, mappings=mappings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vn-register-webportal-embedding-v2'})

In [6]:
client.info()

ObjectApiResponse({'name': 'elasticsearch', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'amh-x3bqSr-tpVcR10jZuw', 'version': {'number': '8.12.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '6185ba65d27469afabc9bc951cded6c17c21e3f3', 'build_date': '2024-02-01T13:07:13.727175297Z', 'build_snapshot': False, 'lucene_version': '9.9.2', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
from tqdm import tqdm

In [8]:
def segment(text):
    _text = re.sub('\.(\s)?\.', '.', text)
    _text = _text.split('.')
    i = 0
    while i + 1 < len(_text):
        _text[i] = _text[i].strip()
        if len(_text[i]) <= 7: 
            _text[i+1] = f'{_text[i]}. {_text[i+1].strip()}'
            _text.pop(i)
        else:
            i += 1
    _text = [_t for _t in _text if len(_t) > 0]
    # print(_text)
    # print([vncorenlp_segmentor.word_segment(_t) for _t in _text])
    return ' '.join([' '.join(vncorenlp_segmentor.word_segment(_t)) for _t in _text])

In [9]:
segment(
    """2.. Bãi bỏ Thông tư số 15/2013/TT-BGTVT ngày 26 tháng 7 năm 2013 của Bộ trưởng Bộ Giao thông vận tải quy định về biểu mẫu, giấy chứng nhận và số kiểm tra an toàn kỹ thuật và bảo vệ môi trường cấp cho tàu biển, phương tiện thủy nội địa và sản phẩm công nghiệp sử dụng cho phương tiện thủy nội địa. """ \
    """3. Các giấy chứng nhận, số an toàn kỹ thuật và bảo vệ môi trường cấp cho tàu biển, phương tiện thủy nội địa và sản phẩm công nghiệp sử dụng cho phương tiện thủy nội địa được cấp trước ngày 01 tháng 9 năm 2017 theo Thông tư số 15/2013/TT-BGTVT ngày 26 tháng 7 năm 2013 của Bộ trưởng Bộ Giao thông vận tải, """ \
    """quy định về biểu mẫu giấy chứng nhận và số kiểm tra an toàn kỹ thuật và bảo vệ môi trường cấp cho tàu biển, phương tiện thủy nội địa và sản phẩm công nghiệp sử dụng cho phương tiện thủy nội địa sẽ tiếp tục có hiệu lực đến ngày hết hiệu lực của các giấy chứng nhận và số này."""
)

'2 . Bãi_bỏ Thông_tư số 15/2013/TT-BGTVT ngày 26 tháng 7 năm 2013 của Bộ_trưởng Bộ Giao_thông vận_tải quy_định về biểu_mẫu , giấy chứng_nhận và số kiểm_tra an_toàn kỹ_thuật và bảo_vệ môi_trường cấp cho tàu_biển , phương_tiện thuỷ nội_địa và sản_phẩm công_nghiệp sử_dụng cho phương_tiện thuỷ nội_địa 3 . Các giấy chứng_nhận , số an_toàn kỹ_thuật và bảo_vệ môi_trường cấp cho tàu_biển , phương_tiện thuỷ nội_địa và sản_phẩm công_nghiệp sử_dụng cho phương_tiện thuỷ nội_địa được cấp trước ngày 01 tháng 9 năm 2017 theo Thông_tư số 15/2013/TT-BGTVT ngày 26 tháng 7 năm 2013 của Bộ_trưởng Bộ Giao_thông vận_tải , quy_định về biểu_mẫu giấy chứng_nhận và số kiểm_tra an_toàn kỹ_thuật và bảo_vệ môi_trường cấp cho tàu_biển , phương_tiện thuỷ nội_địa và sản_phẩm công_nghiệp sử_dụng cho phương_tiện thuỷ nội_địa sẽ tiếp_tục có hiệu_lực đến ngày hết hiệu_lực của các giấy chứng_nhận và số này'

In [11]:
path = '/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/chunks-2'
operations = []
operations_embed = []
for filename in os.listdir(path):
    fp = os.path.join(path, filename)
    print(fp)
    lines = open(fp, 'r').readlines()
    for line in tqdm(lines):
        try:
            chunk = json.loads(line)
            item = {
                "page_url" : chunk["page_url"],
                "category" : chunk["category"],
                # "attachment_url" : chunk["attachment_url"],
                "title": chunk["title"],
                "symbol_number": chunk["symbol_number"],
                "field": chunk["field"],
                "description": chunk["description"],
                "content" : chunk["content"],
                "header": chunk["header"],
            }
            operations.append({"index": {"_index": "vn-register-webportal-v2"}})
            operations.append(item)
            # print(json.dumps(item, indent=4, ensure_ascii=False))
            item_embed = item.copy()
            item_embed['content_vector'] = model.encode(segment(item["content"])).tolist()
            operations_embed.append({"index": {"_index": "vn-register-webportal-embedding-v2"}})
            operations_embed.append(item_embed)
        except Exception as e:
            print(e.args, fp, lines.index(line))


/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/chunks-2/proc-1.txt


  8%|▊         | 302/3786 [00:33<03:46, 15.36it/s]

("Expecting ',' delimiter: line 1 column 880 (char 879)",) /workspace/nlplab/kienvt/KLTN/proc1-extract_storage/chunks-2/proc-1.txt 299


 38%|███▊      | 1439/3786 [03:32<06:22,  6.13it/s] 

("Expecting ',' delimiter: line 1 column 900 (char 899)",) /workspace/nlplab/kienvt/KLTN/proc1-extract_storage/chunks-2/proc-1.txt 1436


...

 44%|████▍     | 1240/2812 [03:03<1:58:39,  4.53s/it]

('JVM exception occurred: Java heap space java.lang.OutOfMemoryError',) /workspace/nlplab/kienvt/KLTN/proc1-extract_storage/chunks-2/proc-3.txt 1239


 75%|███████▌  | 2115/2812 [04:56<02:20,  4.96it/s]  

('Invalid \\escape: line 1 column 914 (char 913)',) /workspace/nlplab/kienvt/KLTN/proc1-extract_storage/chunks-2/proc-3.txt 2112


100%|██████████| 2812/2812 [06:34<00:00,  7.12it/s]


In [12]:
len(operations)

28978

In [22]:
for i in range(len(operations)):
    if operations[i] == {'index': {'_index': 'vn-register-webportal-segment-v2'}}:
        operations[i] = {'index': {'_index': 'vn-register-webportal-v2'}}

In [25]:
client.bulk(index="vn-register-webportal-v2", operations=operations, refresh=True)

ObjectApiResponse({'errors': False, 'took': 6129, 'items': [{'index': {'_index': 'vn-register-webportal-v2', '_id': 'GTokJI4Bt9Ag3h8WfqHl', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'vn-register-webportal-v2', '_id': 'GjokJI4Bt9Ag3h8WfqHl', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'vn-register-webportal-v2', '_id': 'GzokJI4Bt9Ag3h8WfqHl', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'vn-register-webportal-v2', '_id': 'HDokJI4Bt9Ag3h8WfqHl', '_version': 1, 'result': 'created', 'forced_refresh': True, '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_n

In [15]:
bsz = 100
batch_operations_embed = []
for i in range(0, len(operations_embed), 2):
    batch_operations_embed.append(operations_embed[i])
    batch_operations_embed.append(operations_embed[i+1])
    if len(batch_operations_embed) == bsz:
        client.bulk(index="vn-register-webportal-embedding-v2", operations=batch_operations_embed, refresh=True)
        batch_operations_embed.clear()

if len(batch_operations_embed) > 0:
    client.bulk(index="vn-register-webportal-segment-embedding", operations=batch_operations_embed, refresh=True)
    batch_operations_embed.clear()

---

In [ ]:
# pip install -q transformers accelerate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel

checkpoint = "bigscience/mt0-large"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto", 
                                              cache_dir='/workspace/nlplab/kienvt/KLTN/proc1-extract_storage/hf_cache')



In [ ]:
inputs = tokenizer.encode("Translate to English: Je t’aime.", return_tensors="pt").to("cuda")
outputs = model(input_ids=inputs)
# print(tokenizer.decode(outputs[0]))
outputs